badge tests - porting the shields javascript svg generator to python

the generated shields svg for a badge:

```
<svg xmlns="http://www.w3.org/2000/svg" width="110" height="20">
  <linearGradient id="b" x2="0" y2="100%">
    <stop offset="0" stop-color="#bbb" stop-opacity=".1"/>
    <stop offset="1" stop-opacity=".1"/>
  </linearGradient>
  <mask id="a">
    <rect width="110" height="20" rx="3" fill="#fff"/>
  </mask>
  <g mask="url(#a)">
    <path fill="#555" d="M0 0h63v20H0z"/>
    <path fill="#dfb317" d="M63 0h47v20H63z"/>
    <path fill="url(#b)" d="M0 0h110v20H0z"/>
  </g>
  <g fill="#fff" text-anchor="middle" font-family="DejaVu Sans,Verdana,Geneva,sans-serif" font-size="11">
    <text x="31.5" y="15" fill="#010101" fill-opacity=".3">coverage</text>
    <text x="31.5" y="14">coverage</text>
    <text x="85.5" y="15" fill="#010101" fill-opacity=".3">76.8%</text>
    <text x="85.5" y="14">76.8%</text>
  </g>
</svg>
```


In [2]:
import svgwrite
import IPython

In [46]:
# svg badge generator
def generate_svg():
    svg = svgwrite.container.SVG(size=(110,20), **{"xmlns":"http://www.w3.org/2000/svg"})
    
    # add the linear gradient
    linear_gradient = svgwrite.gradients.LinearGradient(end=(0, "100%"), **{"id": "b"})
    linear_gradient.add_stop_color(offset=0, color="#bbb", opacity=".1")
    linear_gradient.add_stop_color(offset=1, opacity=".1")
    svg.add(linear_gradient)
    
    # add the mask
    mask = svgwrite.masking.Mask(**{"id":"a"})
    mask_rect = svgwrite.shapes.Rect(size=(110,20), rx=3, **{"fill": "#fff"})
    mask.add(mask_rect)
    svg.add(mask)
    
    # and the group, with mask
    group = svgwrite.container.Group(**{"mask": "url(#a)"})
    # TODO: i have no idea where those d values come from at this time
    group.add(svgwrite.path.Path(d="M0 0h63v20H0z", **{"fill": "#555"}))
    group.add(svgwrite.path.Path(d="M63 0h47v20H63z", **{"fill": "#dfb317"}))
    group.add(svgwrite.path.Path(d="M0 0h110v20H0z", **{"fill": "url(#b)"}))
    svg.add(group)
    
    # add the text group
    tg = {
        "fill": "#fff", 
        "text-anchor": "middle",
        "font-family": "DejaVu Sans,Verdana,Geneva,sans-serif",
        "font-size": "11"
    }
    text_group = svgwrite.container.Group(**tg)
    text_group.add(svgwrite.text.Text(text="coverage", x=["31.5"], y=["15"], **{"fill": "#010101", "fill-opacity":".3"}))
    text_group.add(svgwrite.text.Text(text="coverage", x=["31.5"], y=["14"]))
    text_group.add(svgwrite.text.Text(text="76.8%", x=["85.5"], y=["15"], **{"fill": "#010101", "fill-opacity":".3"}))
    text_group.add(svgwrite.text.Text(text="76.8%", x=["85.5"], y=["14"]))
    svg.add(text_group)

    return svg.tostring()
    

In [49]:
svg = generate_svg()

# does it match our shields.io example?
print svg

<svg height="20" width="110" xmlns="http://www.w3.org/2000/svg"><defs /><linearGradient id="b" x2="0" y2="100%"><stop offset="0" stop-color="#bbb" stop-opacity=".1" /><stop offset="1" stop-opacity=".1" /></linearGradient><mask id="a"><rect fill="#fff" height="20" rx="3" width="110" x="0" y="0" /></mask><g mask="url(#a)"><path d="M0 0h63v20H0z" fill="#555" /><path d="M63 0h47v20H63z" fill="#dfb317" /><path d="M0 0h110v20H0z" fill="url(#b)" /></g><g fill="#fff" font-family="DejaVu Sans,Verdana,Geneva,sans-serif" font-size="11" text-anchor="middle"><text fill="#010101" fill-opacity=".3" x="31.5" y="15">coverage</text><text x="31.5" y="14">coverage</text><text fill="#010101" fill-opacity=".3" x="85.5" y="15">76.8%</text><text x="85.5" y="14">76.8%</text></g></svg>


In [48]:
IPython.display.SVG(data=svg)

In [50]:
IPython.display.SVG(url="https://rawgit.com/badges/shields/master/coverage.svg")

yes. yes, it does. 

the image matches, there's a bit of svgwrite default cruft in the svg (empty defs element). but that is nothing to worry about today.

let's start building out the dynamic generator.

we need:

1. classes of badge (type of people, type of funding)
2. colors for each type
3. text for each badge option

In [82]:
# dict of types
'''
Funded, FUNding Friday, Testbed
Member, Collaborator
'''
_badge_types = {
    "funded": {"color": "#A1D3E4", "title": "Funded"},
    "funding friday": {"color": "#A1D3E4", "title": "FUNding Friday"},
    "testbed": {"color": "#A1D3E4", "title": "Testbed"},
    "member": {"color": "#388477", "title": "Member"},
    "collaborator": {"color": "#388477", "title": "Collaborator"}
}

_esip = "#3FA1B9"

_pad = 8
_sep = 4
_height = 20

def text_width(text):
    return 7 * len(text)

def calculate_widths(text):
    right_width = _sep + text_width(text) + _pad
    left_width = _pad + text_width('ESIP') + _sep
    return left_width, right_width, left_width + right_width
           
    
def generate_text(text, x, y):
    # return the two text svg elements, text and shadow
    # x and y as strings
    yield svgwrite.text.Text(text=text, x=[str(x)], y=[str(y)], **{"fill": "#010101", "fill-opacity":".3"})
    yield svgwrite.text.Text(text=text, x=[str(x)], y=[str(y-1)])
    
def generate_badge(badge_type):
    badge_type = _badge_types.get(badge_type.lower())
    badge_color = badge_type.get('color')
    badge_name = badge_type.get('title')
    lw, rw, tw = calculate_widths(badge_name)
    
    # build the svg
    svg = svgwrite.container.SVG(size=(tw, 20), **{"xmlns":"http://www.w3.org/2000/svg"})
    
    # add the linear gradient
    linear_gradient = svgwrite.gradients.LinearGradient(end=(0, "100%"), **{"id": "b"})
    linear_gradient.add_stop_color(offset=0, color="#bbb", opacity=".1")
    linear_gradient.add_stop_color(offset=1, opacity=".1")
    svg.add(linear_gradient)
    
    # add the mask
    mask = svgwrite.masking.Mask(**{"id":"a"})
    mask_rect = svgwrite.shapes.Rect(size=(tw,20), rx=3, **{"fill": "#fff"})
    mask.add(mask_rect)
    svg.add(mask)
    
    # and the group, with mask
    group = svgwrite.container.Group(**{"mask": "url(#a)"})
    group.add(svgwrite.shapes.Rect(size=(lw, _height), **{"fill": _esip}))
    group.add(svgwrite.shapes.Rect(size=(rw, _height), insert=(lw, 0), **{"fill": badge_color}))
    group.add(svgwrite.shapes.Rect(size=(tw, _height), **{"fill": "url(#b)"}))
    svg.add(group)
    
    # add the text group
    tg = {
        "fill": "#fff", 
        "text-anchor": "middle",
        "font-family": "DejaVu Sans,Verdana,Geneva,sans-serif",
        "font-size": "11"
    }
    text_group = svgwrite.container.Group(**tg)
    for t in generate_text('ESIP', (lw/2) + 1, 15):
        text_group.add(t)
    for t in generate_text(badge_name, lw + ((rw/2) - 1), 15):
        text_group.add(t)
    svg.add(text_group)
    
    return svg.tostring()

In [83]:
funded = generate_badge('funded')
print funded
print
print

IPython.display.SVG(data=funded)

<svg height="20" width="94" xmlns="http://www.w3.org/2000/svg"><defs /><linearGradient id="b" x2="0" y2="100%"><stop offset="0" stop-color="#bbb" stop-opacity=".1" /><stop offset="1" stop-opacity=".1" /></linearGradient><mask id="a"><rect fill="#fff" height="20" rx="3" width="94" x="0" y="0" /></mask><g mask="url(#a)"><rect fill="#3FA1B9" height="20" width="40" x="0" y="0" /><rect fill="#A1D3E4" height="20" width="54" x="40" y="0" /><rect fill="url(#b)" height="20" width="94" x="0" y="0" /></g><g fill="#fff" font-family="DejaVu Sans,Verdana,Geneva,sans-serif" font-size="11" text-anchor="middle"><text fill="#010101" fill-opacity=".3" x="21" y="15">ESIP</text><text x="21" y="14">ESIP</text><text fill="#010101" fill-opacity=".3" x="66" y="15">Funded</text><text x="66" y="14">Funded</text></g></svg>




In [86]:
friday = generate_badge('funding friday')

IPython.display.SVG(data=friday)

In [87]:
collab = generate_badge('collaborator')

IPython.display.SVG(data=collab)

In [ ]:
# dicking around with the href



s = '''<svg height="20" width="94" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink">
<linearGradient id="b" x2="0" y2="100%">
<stop offset="0" stop-color="#bbb" stop-opacity=".1" />
<stop offset="1" stop-opacity=".1" />
</linearGradient>
<mask id="a">
<rect fill="#fff" height="20" rx="3" width="94" x="0" y="0" />
</mask>
<g mask="url(#a)">
<rect fill="#3FA1B9" height="20" width="40" x="0" y="0" />
<rect fill="#A1D3E4" height="20" width="54" x="40" y="0" />
<rect fill="url(#b)" height="20" width="94" x="0" y="0" />
</g>
<g fill="#fff" font-family="DejaVu Sans,Verdana,Geneva,sans-serif" font-size="11" text-anchor="middle">
<a target="_blank" xlink:href="http://www.esipfed.org"><text fill="#010101" fill-opacity=".3" x="21" y="15">ESIP</text></a>
<text x="21" y="14">ESIP</text>
<text fill="#010101" fill-opacity=".3" x="66" y="15">Funded</text>
<text x="66" y="14">Funded</text>
</g>
</svg>'''


IPython.display.SVG(data=s)